In [ ]:
# Setup: imports and NLP tools
from datasets import load_dataset
import pandas as pd
import numpy as np
import spacy
import nltk
from src.utils import set_seed

set_seed(42)

# Load dataset
print("Loading dataset FutureMa/EvasionBench...")
ds = load_dataset("FutureMa/EvasionBench")
if isinstance(ds, dict):
    ds = ds[list(ds.keys())[0]]
df = ds.to_pandas()
print("Dataset shape:", df.shape)

# Load spaCy model (install if missing)
try:
    nlp = spacy.load("en_core_web_sm")
except Exception:
    print("spaCy model en_core_web_sm not found. Run: python -m spacy download en_core_web_sm")
    nlp = None

# Quick sample
print(df["eva4b_label"].value_counts())
df.sample(5)

# 02 - Linguistic Patterns

**Objectives:**
- Lexical and syntactic analysis by label
- Readability, POS, NER, hedging analysis
- Produce feature candidates for models